In [ ]:
#importing library to print multiple lines in one code shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#importing library to mount my google drive because dataset is uploaded in my google drive
from google.colab import drive
drive.mount('/content/drive/')

#importing library to deal with matrices
import numpy as np

#importing library to deal with csv file dataset
import pandas as pd

#importing library to do some math operations
import math
import random

#importing library used for plotting purpose
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


Mounted at /content/drive/


## **Reading Dataset**

In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/42/SSD sir/solar_energy_prediction_dataset.xlsx')
dataset.head()

,Date,Month,Time,Tw,Tp,Tg,Ta,I(t),Tsky,Xw,Xins,Kins,h (Energy),h (Exergy)
0,15.0,9.0,8.0,33.63,34.40,28.800,24.0,163.5,6.490168,0.018,0.58,0.035,18.636304,0.476462
1,15.0,9.0,9.0,38.70,39.20,33.560,25.6,321.7,7.149884,0.018,0.58,0.035,13.324541,0.456050
2,15.0,9.0,10.0,45.48,46.04,36.200,27.2,523.9,7.830553,0.018,0.58,0.035,23.077021,1.079118
3,15.0,9.0,11.0,53.56,54.14,42.234,29.4,527.3,8.799542,0.018,0.58,0.035,42.970191,2.591368
4,15.0,9.0,12.0,59.66,64.48,46.000,30.5,817.0,9.297985,0.018,0.58,0.035,45.825886,3.275240


## **X-Y Splitting**

In [ ]:
X = dataset[['Date','Month', 'Time', 'Tw', 'Tp', 'Tg', 'Ta', 'I(t)','Tsky','Xw','Xins','Kins']].values
Y = dataset['h (Energy)'].values

## **Modelling**

In [ ]:
import tensorflow as tf
from tensorflow import keras

### Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

### ANN - Keras

In [ ]:
model = keras.Sequential([
                          keras.layers.Dense(20,input_shape=(12,), activation='relu'),
                          keras.layers.Dense(1,activation='linear'),              
])

model.compile(
    optimizer = 'adam',
    loss = 'mean_squared_error',
    metrics = ['accuracy']
)

model.fit(X,Y,epochs = 4000)

Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 3ms/step - loss: 77.0926 - accuracy: 0.0000e+00
Epoch 1502/4000
4/4 [==============================] - 0s 3ms/step - loss: 91.9080 - accuracy: 0.0000e+00
Epoch 1503/4000
4/4 [==============================] - 0s 3ms/step - loss: 78.5435 - accuracy: 0.0000e+00
Epoch 1504/4000
4/4 [==============================] - 0s 3ms/step - loss: 81.2148 - accuracy: 0.0000e+00
Epoch 1505/4000
4/4 [==============================] - 0s 3ms/step - loss: 85.7957 - accuracy: 0.0000e+00
Epoch 1506/4000
4/4 [==============================] - 0s 3ms/step - loss: 77.0779 - accuracy: 0.0000e+00
Epoch 1507/4000
4/4 [==============================] - 0s 4ms/step - loss: 75.7692 - accuracy: 0.0000e+00
Epoch 1508/4000
4/4 [==============================] - 0s 3ms/step - loss: 77.1462 - accuracy: 0.0000e+00
Epoch 1509/4000
4/4 [==============================] - 0s 3ms/step - loss: 74.7715 - accuracy: 0.0000e+00
Epoch 1510/

In [ ]:
print(model.evaluate(X,Y))
Y_pred = model.predict(X)

4/4 [==============================] - 0s 2ms/step - loss: 35.0700 - accuracy: 0.0000e+00
[35.07004928588867, 0.0]


In [ ]:
np.mean((Y_pred - Y)*(Y_pred - Y))

694.3900873924619

### LSTM - Keras

In [ ]:
cols = list(dataset)[:-1]
df_for_training = dataset[cols].astype(float)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [ ]:
### creating features
feature_window_size = 14
prediction_window_size = 1

### for training dataset
X_train = []
Y_train = []
for i in range(feature_window_size, len(df_for_training_scaled)-prediction_window_size+1):
    X_train.append(df_for_training_scaled[i-feature_window_size:i, 0:df_for_training_scaled.shape[1]])
    Y_train.append(df_for_training_scaled[i + prediction_window_size -1:i + prediction_window_size,-1])
X_train, Y_train = np.array(X_train), np.array(Y_train)
print(X_train.shape)
print(Y_train.shape)

(85, 14, 13)
(85, 1)


In [ ]:
# creating a 2 stacked LSTM RNN Model

lstm_regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
lstm_regressor.add(LSTM(units = 64,activation='relu', return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
# lstm_regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
lstm_regressor.add(LSTM(units = 32,activation='tanh', return_sequences = False))
lstm_regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
# lstm_regressor.add(LSTM(units = 50,activation='tanh', return_sequences = True))
# lstm_regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
#lstm_regressor.add(LSTM(units = 50,activation='relu', return_sequences = False))
#lstm_regressor.add(Dropout(0.2))

# Adding the output layer i.e. the optional V matrix wala layer because it is requried here
lstm_regressor.add(Dense(units = Y_train.shape[1]))



In [ ]:
# lstm_regressor.trainable_weights 

### Compiling the RNN
lstm_regressor.compile(optimizer = 'adam', loss = 'mse') 
lstm_regressor.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 14, 64)            19968     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 32,417
Trainable params: 32,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:


### Fitting the LSTM RNN to the Training set
lstm_regressor.fit(X_train, Y_train, epochs = 100, batch_size = 32)

Epoch 1/100
3/3 [==============================] - 2s 13ms/step - loss: 1.0350
Epoch 2/100
3/3 [==============================] - 0s 11ms/step - loss: 0.9464
Epoch 3/100
3/3 [==============================] - 0s 11ms/step - loss: 0.9305
Epoch 4/100
3/3 [==============================] - 0s 11ms/step - loss: 0.8244
Epoch 5/100
3/3 [==============================] - 0s 12ms/step - loss: 0.7465
Epoch 6/100
3/3 [==============================] - 0s 13ms/step - loss: 0.6802
Epoch 7/100
3/3 [==============================] - 0s 12ms/step - loss: 0.6038
Epoch 8/100
3/3 [==============================] - 0s 11ms/step - loss: 0.4643
Epoch 9/100
3/3 [==============================] - 0s 11ms/step - loss: 0.3804
Epoch 10/100
3/3 [==============================] - 0s 11ms/step - loss: 0.2902
Epoch 11/100
3/3 [==============================] - 0s 12ms/step - loss: 0.2752
Epoch 12/100
3/3 [==============================] - 0s 12ms/step - loss: 0.2392
Epoch 13/100
3/3 [==============================]

In [ ]:
y_train_predicted = lstm_regressor.predict(X_train)
y_train_predicted.shape

(85, 1)

In [ ]:
y_train_predicted_copies = np.repeat(y_train_predicted, df_for_training.shape[1], axis = -1)
y_train_predicted_org = scaler.inverse_transform(y_train_predicted_copies)[:,0]
y_train_predicted_org.shape

(85,)

In [ ]:
temp = np.zeros(14)
y_train_predicted_org = np.append(temp,y_train_predicted_org)


In [ ]:
np.mean((y_train_predicted_org[14:]-Y[14:])*(y_train_predicted_org[14:]-Y[14:]))

224.60975719541364

### Ensemble Model : ANN (Keras) + LSTM (Keras)

In [ ]:
dataset

,Date,Month,Time,Tw,Tp,Tg,Ta,I(t),Tsky,Xw,Xins,Kins,h (Energy),h (Exergy)
0,15.0,9.0,8.0,33.630,34.400,28.800,24.0,163.50,6.490168,0.018,0.58,0.035,18.636304,0.476462
1,15.0,9.0,9.0,38.700,39.200,33.560,25.6,321.70,7.149884,0.018,0.58,0.035,13.324541,0.456050
2,15.0,9.0,10.0,45.480,46.040,36.200,27.2,523.90,7.830553,0.018,0.58,0.035,23.077021,1.079118
3,15.0,9.0,11.0,53.560,54.140,42.234,29.4,527.30,8.799542,0.018,0.58,0.035,42.970191,2.591368
4,15.0,9.0,12.0,59.660,64.480,46.000,30.5,817.00,9.297985,0.018,0.58,0.035,45.825886,3.275240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2.0,10.0,14.0,69.000,71.489,61.560,32.2,616.90,10.086091,0.018,0.58,0.035,49.182712,4.316789
95,2.0,10.0,15.0,68.069,74.402,63.250,31.5,485.50,9.758991,0.018,0.58,0.035,35.529124,3.106769
96,2.0,10.0,16.0,58.230,62.562,55.200,30.2,307.30,9.161139,0.018,0.58,0.035,19.485249,1.344351
97,2.0,10.0,17.0,52.150,53.794,50.900,28.5,125.07,8.398590,0.018,0.58,0.035,11.355555,0.673116


In [ ]:
dataset['h_energy_predicted_ann'] = Y_pred
dataset['h_energy_predicted_lstm'] = y_train_predicted_org
dataset['h_energy - h_energy_predicted_ann'] = dataset['h (Energy)'] - dataset['h_energy_predicted_ann']
dataset['h_energy - h_energy_predicted_ann'] = dataset['h_energy - h_energy_predicted_ann'].abs()

dataset['h_energy - h_energy_predicted_lstm'] = dataset['h (Energy)'] - dataset['h_energy_predicted_lstm']
dataset['h_energy - h_energy_predicted_lstm']  = dataset['h_energy - h_energy_predicted_lstm'].abs()
dataset

,Date,Month,Time,Tw,Tp,Tg,Ta,I(t),Tsky,Xw,Xins,Kins,h (Energy),h (Exergy),h_energy_predicted_ann,h_energy_predicted_lstm,h_energy - h_energy_predicted_ann,h_energy - h_energy_predicted_lstm
0,15.0,9.0,8.0,33.630,34.400,28.800,24.0,163.50,6.490168,0.018,0.58,0.035,18.636304,0.476462,21.914938,0.000000,3.278634,18.636304
1,15.0,9.0,9.0,38.700,39.200,33.560,25.6,321.70,7.149884,0.018,0.58,0.035,13.324541,0.456050,20.711863,0.000000,7.387321,13.324541
2,15.0,9.0,10.0,45.480,46.040,36.200,27.2,523.90,7.830553,0.018,0.58,0.035,23.077021,1.079118,28.945969,0.000000,5.868948,23.077021
3,15.0,9.0,11.0,53.560,54.140,42.234,29.4,527.30,8.799542,0.018,0.58,0.035,42.970191,2.591368,50.008755,0.000000,7.038564,42.970191
4,15.0,9.0,12.0,59.660,64.480,46.000,30.5,817.00,9.297985,0.018,0.58,0.035,45.825886,3.275240,48.735161,0.000000,2.909275,45.825886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2.0,10.0,14.0,69.000,71.489,61.560,32.2,616.90,10.086091,0.018,0.58,0.035,49.182712,4.316789,51.241516,28.210152,2.058804,20.972560
95,2.0,10.0,15.0,68.069,74.402,63.250,31.5,485.50,9.758991,0.018,0.58,0.035,35.529124,3.106769,37.299648,20.865911,1.770524,14.663212
96,2.0,10.0,16.0,58.230,62.562,55.200,30.2,307.30,9.161139,0.018,0.58,0.035,19.485249,1.344351,25.018988,13.311035,5.533739,6.174214
97,2.0,10.0,17.0,52.150,53.794,50.900,28.5,125.07,8.398590,0.018,0.58,0.035,11.355555,0.673116,21.558361,8.051116,10.202806,3.304439


In [ ]:
filter = dataset['h_energy - h_energy_predicted_lstm'] >= dataset['h_energy - h_energy_predicted_ann']

dataset.loc[filter, 'h_energy_predicted_temp' ] = dataset['h_energy_predicted_ann']
dataset

,Date,Month,Time,Tw,Tp,Tg,Ta,I(t),Tsky,Xw,Xins,Kins,h (Energy),h (Exergy),h_energy_predicted_ann,h_energy_predicted_lstm,h_energy - h_energy_predicted_ann,h_energy - h_energy_predicted_lstm,h_energy_predicted_temp
0,15.0,9.0,8.0,33.630,34.400,28.800,24.0,163.50,6.490168,0.018,0.58,0.035,18.636304,0.476462,21.914938,0.000000,3.278634,18.636304,21.914938
1,15.0,9.0,9.0,38.700,39.200,33.560,25.6,321.70,7.149884,0.018,0.58,0.035,13.324541,0.456050,20.711863,0.000000,7.387321,13.324541,20.711863
2,15.0,9.0,10.0,45.480,46.040,36.200,27.2,523.90,7.830553,0.018,0.58,0.035,23.077021,1.079118,28.945969,0.000000,5.868948,23.077021,28.945969
3,15.0,9.0,11.0,53.560,54.140,42.234,29.4,527.30,8.799542,0.018,0.58,0.035,42.970191,2.591368,50.008755,0.000000,7.038564,42.970191,50.008755
4,15.0,9.0,12.0,59.660,64.480,46.000,30.5,817.00,9.297985,0.018,0.58,0.035,45.825886,3.275240,48.735161,0.000000,2.909275,45.825886,48.735161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2.0,10.0,14.0,69.000,71.489,61.560,32.2,616.90,10.086091,0.018,0.58,0.035,49.182712,4.316789,51.241516,28.210152,2.058804,20.972560,51.241516
95,2.0,10.0,15.0,68.069,74.402,63.250,31.5,485.50,9.758991,0.018,0.58,0.035,35.529124,3.106769,37.299648,20.865911,1.770524,14.663212,37.299648
96,2.0,10.0,16.0,58.230,62.562,55.200,30.2,307.30,9.161139,0.018,0.58,0.035,19.485249,1.344351,25.018988,13.311035,5.533739,6.174214,25.018988
97,2.0,10.0,17.0,52.150,53.794,50.900,28.5,125.07,8.398590,0.018,0.58,0.035,11.355555,0.673116,21.558361,8.051116,10.202806,3.304439,NaN


In [ ]:
dataset['h_energy_predicted'] = dataset['h_energy_predicted_temp'].combine_first(dataset['h_energy_predicted_lstm'])
dataset

,Date,Month,Time,Tw,Tp,Tg,Ta,I(t),Tsky,Xw,...,Kins,h (Energy),h (Exergy),h_energy_predicted_ann,h_energy_predicted_lstm,h_energy - h_energy_predicted_ann,h_energy - h_energy_predicted_lstm,h_energy_predicted_temp,h_energe_predicted,h_energy_predicted
0,15.0,9.0,8.0,33.630,34.400,28.800,24.0,163.50,6.490168,0.018,...,0.035,18.636304,0.476462,21.914938,0.000000,3.278634,18.636304,21.914938,21.914938,21.914938
1,15.0,9.0,9.0,38.700,39.200,33.560,25.6,321.70,7.149884,0.018,...,0.035,13.324541,0.456050,20.711863,0.000000,7.387321,13.324541,20.711863,20.711863,20.711863
2,15.0,9.0,10.0,45.480,46.040,36.200,27.2,523.90,7.830553,0.018,...,0.035,23.077021,1.079118,28.945969,0.000000,5.868948,23.077021,28.945969,28.945969,28.945969
3,15.0,9.0,11.0,53.560,54.140,42.234,29.4,527.30,8.799542,0.018,...,0.035,42.970191,2.591368,50.008755,0.000000,7.038564,42.970191,50.008755,50.008755,50.008755
4,15.0,9.0,12.0,59.660,64.480,46.000,30.5,817.00,9.297985,0.018,...,0.035,45.825886,3.275240,48.735161,0.000000,2.909275,45.825886,48.735161,48.735161,48.735161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2.0,10.0,14.0,69.000,71.489,61.560,32.2,616.90,10.086091,0.018,...,0.035,49.182712,4.316789,51.241516,28.210152,2.058804,20.972560,51.241516,51.241516,51.241516
95,2.0,10.0,15.0,68.069,74.402,63.250,31.5,485.50,9.758991,0.018,...,0.035,35.529124,3.106769,37.299648,20.865911,1.770524,14.663212,37.299648,37.299648,37.299648
96,2.0,10.0,16.0,58.230,62.562,55.200,30.2,307.30,9.161139,0.018,...,0.035,19.485249,1.344351,25.018988,13.311035,5.533739,6.174214,25.018988,25.018988,25.018988
97,2.0,10.0,17.0,52.150,53.794,50.900,28.5,125.07,8.398590,0.018,...,0.035,11.355555,0.673116,21.558361,8.051116,10.202806,3.304439,NaN,8.051116,8.051116


In [ ]:
np.mean((dataset['h_energy_predicted'] - dataset['h (Energy)'])*(dataset['h_energy_predicted'] - dataset['h (Energy)']))

25.205098778779615